In [2]:
"""Global functions and variables"""

import pymysql

def open_conn():
    """open the connection before each test case"""
    conn = pymysql.connect(user='public', password='ece656yelp',
                                   host='maindb.czbva1am4d4u.us-east-2.rds.amazonaws.com',
                                   database='yelp_db')
    return conn

def close_conn(conn):
    """close the connection after each test case"""
    conn.close()

def executeQuery(conn, query, commit=False):
    """ fetch result after query"""
    cursor = conn.cursor()
    query_num = query.count(";")
    if query_num > 1:
        for result in cursor.execute(query, params=None, multi=True):
            if result.with_rows:
                result = result.fetchall()
    else:
        cursor.execute(query)
        result = cursor.fetchall()
    # we commit the results only if we want the updates to Can't leave a review dated before account creationthe database
    # to persist.
    if commit:
        conn.commit()
    else:
        conn.rollback()
    # close the cursor used to execute the query
    cursor.close()
    return result

yelp_conn = open_conn()

## Part I. Data cleaning

#### 1. Check that no review is from the future or before Yelp's founding
FINISHED.
Result: No such reviews.

In [9]:
query_1 = "SELECT * FROM review WHERE date <= '2004-07-01' OR date >= '2018-01-01'";

result_1 = executeQuery(yelp_conn, query_1)
result_1

()

#### 2. Can't leave a review dated before account creation

In [14]:
query_2 =  "SELECT user.id, user.yelping_since AS Date_of_yelping, review.date AS Date_of_Review\
            FROM (user INNER JOIN review ON user.id = review.user_id)\
            WHERE user.yelping_since < review.date\
            GROUP BY user.id;"

result_2 = executeQuery(yelp_conn, query_2)

In [19]:
len(result_2)

1061718

This weird because the total count of users in `user` table is just 1183362.

In [18]:
print(result_2[0:200])

(('---1lKK3aKOuomHnwAkAow', datetime.datetime(2007, 6, 4, 0, 0), datetime.datetime(2012, 9, 7, 0, 0)), ('---94vtJ_5o_nikEs6hUjg', datetime.datetime(2016, 5, 27, 0, 0), datetime.datetime(2016, 8, 17, 0, 0)), ('---cu1hq55BP9DWVXXKHZg', datetime.datetime(2009, 4, 18, 0, 0), datetime.datetime(2014, 11, 19, 0, 0)), ('---fhiwiwBYrvqhpXgcWDQ', datetime.datetime(2011, 4, 20, 0, 0), datetime.datetime(2013, 5, 1, 0, 0)), ('---PLwSf5gKdIoVnyRHgBA', datetime.datetime(2015, 7, 31, 0, 0), datetime.datetime(2017, 2, 13, 0, 0)), ('---udAKDsn0yQXmzbWQNSw', datetime.datetime(2014, 7, 12, 0, 0), datetime.datetime(2017, 7, 14, 0, 0)), ('--0kuuLmuYBe3Rmu0Iycww', datetime.datetime(2010, 3, 8, 0, 0), datetime.datetime(2013, 9, 12, 0, 0)), ('--0RtXvcOIE4XbErYca6Rw', datetime.datetime(2013, 5, 30, 0, 0), datetime.datetime(2013, 8, 7, 0, 0)), ('--0sXNBv6IizZXuV-nl0Aw', datetime.datetime(2013, 1, 9, 0, 0), datetime.datetime(2016, 3, 12, 0, 0)), ('--0WZ5gklOfbUIodJuKfaQ', datetime.datetime(2013, 2, 19, 0, 0), dat

#### 3. Can't be elite in a year before their account was made

In [22]:
query_3 =  "SELECT user.id, user.yelping_since AS Date_of_yelping, elite_years.year AS Year_of_Elite\
            FROM (user INNER JOIN elite_years ON user.id = elite_years.user_id)\
            WHERE YEAR(user.yelping_since) < elite_years.year\
            GROUP BY user.id;"

result_3 = executeQuery(yelp_conn, query_3)

In [23]:
len(result_3)

55493

#### 4. Can't checkin outside open hours

#### 5. `User.number_reviews` cannot be less than the sum of their review